# Part 1

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
request = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
bs = BeautifulSoup(request.content,'lxml')
table = bs.find_all('table')[0]
temp_df = pd.read_html(str(table))
df=pd.DataFrame(temp_df[0])

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
df=df[df.Borough != 'Not assigned']
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [4]:
idx_list = df.loc[df['Neighbourhood'] == "Not assigned"].index
for i in idx_list:
    df.iloc[i, 2] = df.iloc[i, 1] 
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')
df.head()

,Postcode,Borough,Neighbourhood
0,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,..."
1,M8W,Etobicoke,"Alderwood,Long Branch"
2,M3B,North York,Don Mills North
3,M4N,Central Toronto,Lawrence Park
4,M5H,Downtown Toronto,"Adelaide,King,Richmond"


In [6]:
df.shape

(103, 3)

# Part 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

In [7]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [8]:
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Use the Geocoder package or the csv file to create the following dataframe:

In [9]:
df_toronto = df.join(df_geo.set_index('Postal Code'), on='Postcode')
df_toronto.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,M3B,North York,Don Mills North,43.745906,-79.352188
3,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
4,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
5,M6B,North York,Glencairn,43.709577,-79.445073
6,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
7,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
8,M4C,East York,Woodbine Heights,43.695344,-79.318389
9,M9P,Etobicoke,Westmount,43.696319,-79.532242


# Part 3

### Import libraries

In [10]:
!pip install geopy folium
import geocoder
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Rearrange dataframe

In [11]:
df=df_toronto[df_toronto['Borough'].str.contains('Toronto')]
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
3,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
4,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
6,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
10,M5N,Central Toronto,Roselawn,43.711695,-79.416936


### Make dataframe available to fit

In [12]:
temp_list = ['Postcode', 'Borough','Neighbourhood']
df_dropped = df.drop(temp_list, 1)
df_dropped.head()

,Latitude,Longitude
0,43.686412,-79.400049
3,43.728020,-79.388790
4,43.650571,-79.384568
6,43.662696,-79.400049
10,43.711695,-79.416936


### K-means Clustering and labeling to dataframe

In [13]:
clusters = 7
kmeansclustering = KMeans(n_clusters=clusters).fit(df_dropped)

df.insert(0, 'Cluster', kmeansclustering.labels_)
df.head()

,Cluster,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,6,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
3,4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
4,0,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
6,6,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
10,4,M5N,Central Toronto,Roselawn,43.711695,-79.416936


### Create the map

In [14]:
address = 'Toronto'
nom_geolocator = Nominatim(user_agent="toronto_neighbor_explorer")
location = nom_geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

geomap_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for latit, longit, borough, neighborhood, cluster in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood'], df['Cluster']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latit, longit],
        radius=10,
        popup=label,
        color= rainbow[cluster-1],
        fill=True,
        fill_color= rainbow[cluster-1],
        fill_opacity=0.5,
        parse_html=False).add_to(geomap_toronto)  
    
geomap_toronto